In [1]:
import requests
import pandas as pd
import json

In [2]:
#On définie une liste comportant les villes que l'on souhaite étudier

cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
#Test requête API nominatim avec la ville de Paris saisie manuellement, découverte du filtre country

url = "https://nominatim.openstreetmap.org/search"
payload = {
    'city' : 'Paris',
    'country' : 'France',
    'format':'json'
    }

gps_coord = requests.get(url, params = payload)
gps_coord

<Response [200]>

In [4]:
gps_coord.json()

[{'place_id': 115047027,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 7444,
  'lat': '48.8588897',
  'lon': '2.3200410217200766',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 15,
  'importance': 0.8317101715588673,
  'addresstype': 'suburb',
  'name': 'Paris',
  'display_name': 'Paris, Île-de-France, France métropolitaine, France',
  'boundingbox': ['48.8155755', '48.9021560', '2.2241220', '2.4697602']},
 {'place_id': 115134379,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 71525,
  'lat': '48.8534951',
  'lon': '2.3483915',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 12,
  'importance': 0.8317101715588673,
  'addresstype': 'city',
  'name': 'Paris',
  'display_name': 'Paris, Île-de-France, France métropolitaine, France',
  'boundingbox': ['48.8155755', '48.9021560', '2.2241220', '2.4697

In [5]:
#Test coordonnés auprès du résultat de la requête

cities_lat = gps_coord.json()[0]['lat']
print(cities_lat)
cities_lon = gps_coord.json()[0]['lon']
print(cities_lon)

48.8588897
2.3200410217200766


In [6]:
#Nous itérons maintenant sur la liste cities pour extraire toutes les informations des villes cible

target_coord = []

for city in cities :
    url = "https://nominatim.openstreetmap.org/search"
    payload = {
    'city' : city ,
    'country' : 'France',
    'format':'json'
    }

    gps_coord = requests.get(url, params = payload)
    target_coord.append(gps_coord.json())

In [7]:
print(target_coord)


[[{'place_id': 276833152, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 211285890, 'lat': '48.6359541', 'lon': '-1.511459954959514', 'class': 'place', 'type': 'islet', 'place_rank': 20, 'importance': 0.45543655678157396, 'addresstype': 'islet', 'name': 'Mont Saint-Michel', 'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France', 'boundingbox': ['48.6349172', '48.6370310', '-1.5133292', '-1.5094796']}, {'place_id': 101830509, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1581196576, 'lat': '43.8496443', 'lon': '7.0938927', 'class': 'natural', 'type': 'peak', 'place_rank': 18, 'importance': 0.30000999999999994, 'addresstype': 'peak', 'name': 'Mont Saint-Michel', 'display_name': "Mont Saint-Michel, Les Ferres, Grasse, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06

In [8]:
#Création d'une liste contenant uniquement le nom des villes cible et leurs coordonnés GPS

city_coord = []

for coord in target_coord :
    lat = coord[0]['lat']
    lon = coord[0]['lon']
    name = coord[0]['name']

    city_coord.append({"name": name,"lat":lat,"lon":lon})


In [9]:
print(city_coord)

[{'name': 'Mont Saint-Michel', 'lat': '48.6359541', 'lon': '-1.511459954959514'}, {'name': 'Saint-Malo', 'lat': '48.649518', 'lon': '-2.0260409'}, {'name': 'Bayeux', 'lat': '49.2764624', 'lon': '-0.7024738'}, {'name': 'Le Havre', 'lat': '49.4938975', 'lon': '0.1079732'}, {'name': 'Rouen', 'lat': '49.4404591', 'lon': '1.0939658'}, {'name': 'Paris', 'lat': '48.8588897', 'lon': '2.3200410217200766'}, {'name': 'Amiens', 'lat': '49.8941708', 'lon': '2.2956951'}, {'name': 'Lille', 'lat': '50.6365654', 'lon': '3.0635282'}, {'name': 'Strasbourg', 'lat': '48.584614', 'lon': '7.7507127'}, {'name': 'Château du Haut-Kœnigsbourg', 'lat': '48.2495226', 'lon': '7.3454923'}, {'name': 'Colmar', 'lat': '48.0777517', 'lon': '7.3579641'}, {'name': 'Eguisheim', 'lat': '48.0447968', 'lon': '7.3079618'}, {'name': 'Besançon', 'lat': '47.2380222', 'lon': '6.0243622'}, {'name': 'Dijon', 'lat': '47.3215806', 'lon': '5.0414701'}, {'name': 'Annecy', 'lat': '45.8992348', 'lon': '6.1288847'}, {'name': 'Grenoble', 'l

In [10]:
#Mise en forme data frame Pandas

df = pd.DataFrame(city_coord)
display(df)

,name,lat,lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658
5,Paris,48.8588897,2.3200410217200766
6,Amiens,49.8941708,2.2956951
7,Lille,50.6365654,3.0635282
8,Strasbourg,48.584614,7.7507127
9,Château du Haut-Kœnigsbourg,48.2495226,7.3454923


In [11]:
#test d'une boucle pour récupérer les coordonnées auprès du dataframe

#for i in range(0,len(df)):
#  print(df['lat'][i])

In [12]:
#Test requête API
#url = "https://api.openweathermap.org/data/2.5/forecast"
#payload = {"lat":"46.1591126",
#           "lon":"-1.1520434",
#           "exclude":"minutely,hourly,daily",
#           "units":"metric",
#           "appid":"3980a5f4b8c014468102d19a16bf8c86"
#}

#r = requests.get(url, params=payload)
#r.json()

#écrire lignes de codes pour filtrer les infos !!!!

In [13]:
#Echec premier test

#r_df = pd.DataFrame(r)
#r_df.head()

In [28]:
from datetime import datetime

In [42]:
#On initialise une liste qui contiendra les villes et les données climatiques pertinentes

city_weather = []

#On boucle sur les éléments du dataframe des villes cible et leurs coordonnées pour extraire de l'API les données climatiques relatives à chacune d'entre elles

for i in range(0,len(df)):

  url = "https://api.openweathermap.org/data/2.5/forecast"
  payload = {"lat":df['lat'][i],
           "lon":df['lon'][i],
           "exclude":"current,minutely,hourly,alerts",
           "units":"metric",
           "appid":"3980a5f4b8c014468102d19a16bf8c86"
  }

  response = requests.get(url, params=payload)
  data = response.json()

#Nouvelle boucle permettant d'itérer sur les différentes dates

  for forecast in data["list"]:

    time = forecast["dt"]
    date_time = datetime.utcfromtimestamp(time)

#Création d'un dictionnaire qui permette de cibler les informations pertinentes relevées par la boucle dans l'API

    weather_data = {
      "time" : date_time.strftime("%Y-%m-%d %H"),
      "city": df["name"][i],
      "temperature": forecast["main"]["temp"],
#      "weather": data["list"][0]["weather"][0]["main"],
#      "clouds": data["list"][0]["clouds"]["all"]
    }

#Ajout des informations pertinentes à liste initialisée au départ

    city_weather.append(weather_data)


In [43]:
print(city_weather)

[{'time': '2024-04-03 18', 'city': 'Mont Saint-Michel', 'temperature': 14.26}, {'time': '2024-04-03 21', 'city': 'Mont Saint-Michel', 'temperature': 13.5}, {'time': '2024-04-04 00', 'city': 'Mont Saint-Michel', 'temperature': 11.91}, {'time': '2024-04-04 03', 'city': 'Mont Saint-Michel', 'temperature': 11.79}, {'time': '2024-04-04 06', 'city': 'Mont Saint-Michel', 'temperature': 11.99}, {'time': '2024-04-04 09', 'city': 'Mont Saint-Michel', 'temperature': 13.56}, {'time': '2024-04-04 12', 'city': 'Mont Saint-Michel', 'temperature': 15.12}, {'time': '2024-04-04 15', 'city': 'Mont Saint-Michel', 'temperature': 12.88}, {'time': '2024-04-04 18', 'city': 'Mont Saint-Michel', 'temperature': 13.25}, {'time': '2024-04-04 21', 'city': 'Mont Saint-Michel', 'temperature': 12.2}, {'time': '2024-04-05 00', 'city': 'Mont Saint-Michel', 'temperature': 11.45}, {'time': '2024-04-05 03', 'city': 'Mont Saint-Michel', 'temperature': 13.84}, {'time': '2024-04-05 06', 'city': 'Mont Saint-Michel', 'temperatu

In [25]:
#print de la requête API au format json.dumps pour mieux identifier l'arborescence et cibler les informations à filtrer dans weather_data

print(json.dumps(data, indent=6))

{
      "cod": "200",
      "message": 0,
      "cnt": 40,
      "list": [
            {
                  "dt": 1712167200,
                  "main": {
                        "temp": 14.46,
                        "feels_like": 14.18,
                        "temp_min": 13.2,
                        "temp_max": 14.46,
                        "pressure": 1015,
                        "sea_level": 1015,
                        "grnd_level": 1015,
                        "humidity": 85,
                        "temp_kf": 1.26
                  },
                  "weather": [
                        {
                              "id": 804,
                              "main": "Clouds",
                              "description": "overcast clouds",
                              "icon": "04d"
                        }
                  ],
                  "clouds": {
                        "all": 100
                  },
                  "wind": {
                        "speed": 

In [44]:
#On affiche le dataframe avec les informations climatiques pertinentes

df2 = pd.DataFrame(city_weather)
display(df2)

,time,city,temperature
0,2024-04-03 18,Mont Saint-Michel,14.26
1,2024-04-03 21,Mont Saint-Michel,13.50
2,2024-04-04 00,Mont Saint-Michel,11.91
3,2024-04-04 03,Mont Saint-Michel,11.79
4,2024-04-04 06,Mont Saint-Michel,11.99
...,...,...,...
1395,2024-04-08 03,La Rochelle,12.81
1396,2024-04-08 06,La Rochelle,13.51
1397,2024-04-08 09,La Rochelle,15.37
1398,2024-04-08 12,La Rochelle,17.08


In [52]:
df3 = df2.groupby("city")["temperature"].mean()
print(df3)


city
Aigues-Mortes                  15.29425
Aix-en-Provence                17.29400
Amiens                         13.62800
Annecy                         14.86875
Avignon                        16.72075
Bayeux                         12.94725
Bayonne                        16.74875
Besançon                       15.26750
Biarritz                       16.40400
Bormes-les-Mimosas             14.96150
Carcassonne                    15.36700
Cassis                         16.99250
Château du Haut-Kœnigsbourg    12.79125
Collioure                      15.89725
Colmar                         15.87200
Dijon                          14.13050
Eguisheim                      16.05175
Gorges du Verdon               13.05300
Grenoble                       17.22400
La Rochelle                    14.11525
Le Havre                       12.84275
Lille                          13.85850
Lyon                           16.61525
Marseille                      16.94200
Mont Saint-Michel              12.9